# COMP9318 Project
## Exploratory Data Analysis

In [1]:
import helper as h
import pandas as pd
import sklearn
import pandas2arff

Read in word data:

In [2]:
data_loc = 'asset/training_data.txt'
words = h.get_words(data_loc)

Convert to data frame

In [3]:
words_df = pd.DataFrame([word.__dict__ for word in words])

In [4]:
words_df.columns

Index(['consonant_map', 'first_letter_index', 'pn_list', 'primary_stress_idx',
       'primary_stress_map', 'pronunciation', 'secondary_stress_map',
       'type_tag', 'vector_map', 'vowel_map', 'word'],
      dtype='object')

Get pos_tags

In [9]:
columns = ['word','type_tag','first_letter_index','primary_stress_idx']
words_features  = words_df[columns]
words_features[words_features['first_letter_index']==11].describe()


,first_letter_index,primary_stress_idx
count,1518.0,1518.000000
mean,11.0,2.655468
std,0.0,1.126422
min,11.0,2.000000
25%,11.0,2.000000
50%,11.0,2.000000
75%,11.0,3.000000
max,11.0,8.000000


In [ ]:
%matplotlib inline

for pos_tag in pos_tags:
    temp_df = words_df[words_df['type_tag'] ==pos_tag]
    temp_df.plot(temp_df.primary_stress_idx,kind='scatter')